# BUAD 313 - Spring 2025 - Assignment 2

Notes:
 - You may work in teams of up to 3.  Submit one assignment for the three of you, but please ensure it has all 3 of your names and @usc.edu emails.
 - You must submit your work as a .ipynb file (jupyter notebook). The grader has to be able to run your notebook. Code that doesn't run gets zero points.  A Great way to check that your code runs is to Select "Clear All Outputs", "Restart" and then "Run All" and make sure it still shows all your answer properly!
 - Use the existing sections below to submit your answers below.  You can add additional Python/markdown cells to describe and explain your solution, but keep it tidy.  Consider formatting some of your markdown cells for the grader.  [Markdown Guide](https://www.markdownguide.org/basic-syntax/)

The deadline for this assignment is **11:59 PM Pacific Time on Friday February 21, 2024**. Late submissions will not be accepted.

Below are the standard Python packages that we use for optimization models in this course. By running this next Python cell, you will have these packages available to use in all your answers contained in this file.

In [1]:
import numpy as np
from gurobipy import Model, GRB, quicksum
import pandas as pd

## Team Names and Emails:
 <font color="blue">**(Edit this cell)**</font>
 - William Jou: wcjou@usc.edu

## Question 1 (45 Points):  Portfolio Allocation Revisited

In this problem, we are revisiting the portfolio allocation problem we started developing in Session 9.  You may want to review that lecture and the mathematical formulation we developed. As a reminder, here was the formulation for the base model from class:

<img src="PortfolioProblem.png" alt="Base Portfolio Allocation Model Model" width="400" height=auto>


Assume a target return of .01.  

Our data for this problem are available on brightspace and include:
- monthly_ret_simple.csv
- asset_metadata.csv

I did all the data-wrangling for you (because I'm a nice guy).  Below I load up monthly_ret_simple.csv into a monthly_returns_dict like we did in class.  I also load up asset_metadata.csv into a dictionary called asset_metadata_dict.  You'll probably need that dictionary later in the question.

In [3]:
#read monthly_ret_simple.csv using numpy, but ignore the first row and ignore first column
monthly_returns = np.genfromtxt('monthly_ret_simple.csv', delimiter=',', skip_header=True)[:,1:]

#read in just the first row of monthly_ret_simple.csv, ignore first column and label as tickers
tickers = np.genfromtxt('monthly_ret_simple.csv', delimiter=',', max_rows=1, dtype=str) [1:]

#read in just the first column of monthly_ret_simple.csv, ignoring the first row and label as dates
dates = np.genfromtxt('monthly_ret_simple.csv', delimiter=',', skip_header=True, usecols=0, dtype=str)

#convert monthly_returns into a dictionary where
# the keys are pairs of (date, ticker) and values are the return
monthly_returns_dict = { (dates[i], tickers[j]) : monthly_returns[i,j] for i in range(len(dates)) for j in range(len(tickers)) }

#compute a dictionary of the average returns for each asset
average_returns = { ticker : np.mean([ monthly_returns_dict[(date, ticker)] for date in dates ]) for ticker in tickers }

In [4]:
#read asset_metadata.csv into a dataframe and convert to a dictionary
asset_metadata = pd.read_csv('asset_metadata.csv')

# Convert to a dictionary where (ticker, column_label) -> value
asset_dict = {(row["Ticker"], col): row[col] for _, row in asset_metadata.iterrows() for col in asset_metadata.columns if col != "Ticker"}

categories = [col for col in asset_metadata.columns if col != "Ticker"]

### Part a) (10 points):
One of the criticisms brought up in class was that absolute deviation penalizes being above the expected return the same way it penalizes being below the expected return.  That seems silly because being above the expected return is a good thing.  

One way to address this problem is to use semi-deviation.  The monthly semi-deviation of a portfolio is
 - 0 if the return of the portfolio in that month is above its expected return.
 - the expected return of the portfolio minus the return of the portfolio in that month, otherwise.

Thus, there is no penalty if the portfolio outperforms the expected return, but there is a penalty if it underperforms.

Modify our base model to minimize the average semi-deviation over the dataset.  Write out a full linear optimization formulation for your new model (decision varaibles, constraints, and objective).  You may add/remove variables and constraints from the base model, and/or change the objective.  All variables should be continuous.  Be sure to explain any new variables, constraints or objective in words and in mathematical formulas.

## PART A ANSWER

### DECISION VARIABLES
- let x_t = the weights of each ticker t in the portfolio
- let dev_d = the deviations of the returns on each date d

### CONSTRAINTS
- The sum of all the weights of the tickers must be equal to 1 as we must use up all of our wealth. For all tickers t, sum(x_t) = 1
- Our actual return must be greater than or equal to our target return. For all tickers t, x_t * average_return_t >= target return
- For every date d, the deviation has to be greater than or equal to the expected return - the actual return
- for every date d, the deviation cannot be less than 0

### OBJECTIVE FUNCTION
- min(sum(dev_d for all d) / total number of periods)



In [ ]:
m = Model('portfolio')
target_return = 0.01

# DECISION VARIABLES

x = m.addVars(tickers, lb=0, vtype=GRB.CONTINUOUS)
dev = m.addVars(dates, lb=0, vtype=GRB.CONTINUOUS)

# CONSTRAINTS

# Investing all the wealth (The portfolio weights have to add up to 1)
m.addConstr(quicksum(x[ticker] for ticker in tickers) == 1)

# Semi-Deviation Constraint
for date in dates:
    actual_return = quicksum(x[ticker] * monthly_returns_dict[date, ticker] for ticker in tickers)
    expected_return = quicksum(x[ticker] * average_returns[ticker] for ticker in tickers)
    m.addConstr(dev[date] >= expected_return - actual_return)
    # m.addConstr(dev[date] >= 0) not needed, since set lower bound of dev to 0


# We must achieve target greater than or equal to our target return
m.addConstr(quicksum(average_returns[ticker] * x[ticker] for ticker in tickers) >= target_return)

# OBJECTIVE FUNCTION

m.setObjective(quicksum(dev[date] for date in dates) / len(dates), GRB.MINIMIZE)

# OPTIMIZATION

m.optimize()

# Printing optimal weights
for ticker in tickers:
    print(f"{ticker}: {x[ticker].x:.4f}")



Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 265K, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 28 rows, 48 columns and 642 nonzeros
Model fingerprint: 0xc8796982
Coefficient statistics:
  Matrix range     [5e-05, 1e+00]
  Objective range  [4e-02, 4e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e+00]
Presolve time: 0.00s
Presolved: 28 rows, 48 columns, 642 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.238629e-01   0.000000e+00      0s
      30    1.2623371e-02   0.000000e+00   0.000000e+00      0s

Solved in 30 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.262337120e-02
AAPL: 0.0000
ARKK: 0.0000
BABA: 0.0000
BITO: 0.0000
EEM: 0.0000
EWJ: 0.1139
FSLR: 0.0191
GLD: 0.5733
GRN: 0.0407
HASI: 0.0000
ICLN: 0.0000
LIT: 0.0000
MSFT: 0.

### Part b) (10 points)
The client decides this whole semi-deviation idea is too complicated for their taste.  They propose a simpler optimization model similar to the base model. They want to change the objective to maximize the worst monthly return the portfolio earns over the whole dataset, but otherwise keep the other portions of the problem the same (invest all the wealth, don't short-sell, achieve a target return of .01).  

Write a linear optimization formulation for this new problem (decision variables, constraints, and objective).  Be clear and explain mathematics in words where appropriate. Your formulation should only use continuous decision variables.

**Hint** Even though the client only wants to change the objective, you might need to change the constraints and variables too to achieve what they want


## PART B ANSWER

### DECISION VARIABLES:
- let x_t = the weights of each ticker t in the portfolio
- let r_min = the lowest monthly return

### CONSTRAINTS:
- The sum of all the weights of the tickers must be equal to 1 as we must use up all of our wealth. For all tickers t, sum(x_t) = 1
- Our actual return must be greater than or equal to our target return. For all tickers t, x_t * average_return_t >= target return
- All of the monthly returns have to be greater than or equal to the lowest monthly return. For all dates d, and all tickers t, x_t * monthly_return_d,t >= r_min

### OBJECTIVE FUNCTION:
- max(r_min): we want to maximize the lowest monthly return


### Part c) 10 points
Using your formulation in part b, code up your model in Gurobi and solve it.  (Use a target_return of .01).  

Include your Python code for the model in one or more Python cells.  Your code should print out the optimal value and optimal solution from your model as its last step. Be sure to label which is which and what the units are! Code that does not run earns no credit!

In [21]:
m = Model('portfolio')
target_return = 0.01

# DECISION VARIABLES
x = m.addVars(tickers, lb=0, vtype=GRB.CONTINUOUS)
r_min = m.addVar(vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, name='Lowest_Monthly_Return')

# CONSTRAINTS

m.addConstr(quicksum(x[ticker] for ticker in tickers) == 1)
m.addConstr(quicksum(x[ticker] * average_returns[ticker] for ticker in tickers) >= target_return)

for date in dates:
    m.addConstr(quicksum(x[ticker] * monthly_returns_dict[date, ticker] for ticker in tickers) >= r_min)

# OBJECTIVE FUNCTION

m.setObjective(r_min, GRB.MAXIMIZE)

m.optimize()

# Print the optimal portfolio weights for each ticker
for ticker in tickers:
    print(f"Optimal weight for {ticker}: {x[ticker].X}")

# Print the optimal value of the worst return (r_min)
print(f"Optimal worst monthly return: {r_min.X}")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 265K, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 28 rows, 23 columns and 642 nonzeros
Model fingerprint: 0x87203428
Coefficient statistics:
  Matrix range     [9e-05, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e+00]
Presolve time: 0.01s
Presolved: 28 rows, 23 columns, 642 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
       7   -3.4811432e-02   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective -3.481143246e-02
Optimal weight for AAPL: 0.0
Optimal weight for ARKK: 0.0
Optimal weight for BABA: 0.0
Optimal weight for BITO: 0.0
Optimal weight for EEM: 0.0
Optimal weight f

### Part d) (15 points)

The client saw your initial work in class and has now articulated a variety of additional constraints on the portfolio.  For each of the constraints below, write (paper and pencil) how you would represent the constraint mathematically as one or more linear constraints in our model.  You may add additional auxiliary variables if you need them, but clearly define them and what they should mean.  You may use the indexed data originally loaded at the top of this question.  You may also define new Index Sets if you so choose.

Add a Markdown Cell directly below this cell with your answer.  You do NOT need to code these constraints in your model.  Alternatively, if typing in markdown isn't your thing, you can write this on paper and pencil (clearly) and take a photo, and include the photo in markdown the way I've down above for the formulation.  Just make sure you remember to upload the photo with your assignment else you won't get credit!

Finally, be sure to label each constraint (same way in question) so we know which one is which!

1. (China Tariff Concerns) The client is concerned about the impending trade war with China.  They want at most 10% of the portfolio invested in Chinese assets.
1. (ESG Requirement) No more than 30% of the portfolio can be invested in assets with a "low" ESG rating.  
1. (Liquidity Requirement) The ratio of "Low" liquidity assets to "High" liquidy assets should be no more than 10%.
1. (Commodities Minimum) The amount invested in the Asset Class "Commodities" should be at least 10% of the amount invested in the Asset Class "Equities."
1. (Diversifying Equities) Within the investments in Equities, a third of them should be Small Cap, a third should be Mid Cap, and a third should be Large Cap.  
1. (No Tesla) The client does not want to invest in Tesla (TSLA) at all.

**Hint 1:**  DO THIS WITH PAPER AND PENCIL BEFORE TYPING ANYTHING.  

**Hint 2:** I"m going to solve the first part of the problem for you.  Read my answer and mimic it for remaining parts.

**China Tariff Concerns** Let C be the set of tickers $i$ where asset_data[$i$, "Region] = "China."  (C is an Index Set.)  Then, the constraint can be written as
- sum over i in C of x[i] <= .3.  
  
Or, if I want to make it look a bit prettier, I can write
 - $ \sum_{i \text{ in C}} x_i \leq .1 $

(Either answer is fine.)

## Question 2 (26 Points): After that last question, I could use a ...

Trojan Microbrewers brews four types beers: Light, Dark, Ale, and Premium (abreviated L, D, A, P).  Beer is made from 3 main ingredients: malt, hops and yeast.  Each of the different beers requires different amounts of each ingredient to make one beer, plus a lot of other minor ingredients like artificial flavors and preservatives.

Trojan Microbrewers currently has some inventory of malt, hops and yeast (in pounds), but it has an essentially unlimited supply of artificial flavors and preservatives.

Finally, each beer has its own revenue per bottle sold (in dollars).

Your colleague formulated a linear optimization model to maximize the revenue of Trojan Microbrewers
subject to the constraints on the availability of current inventory of main ingredients, taking into account the different recipes. (They assumed we're not buying any more main ingredients at the moment.)
                                                                      
After formulating the model, they solved it and computed the following sensitivity analysis table, but forgot to share the actual formulation with you.

<img src="beer_sensitivity2.png" alt="Sensitivity Table for the Beer model" width="800" height="auto">


#### Part a (5 points)
From just the sensitivity table, can you say what the optimal objective value is?  If so, be sure to explain your answer and how you deduced it from the table for full credit and give units. If not, explain why not and give the best bounds you can.

#### Part b (3 points)
From just the sensitivity table, can you say what the current inventory of Malt, Hops and yeast is?  If so, provide them and explain how you deduced this from the table.  If not, explain why not and provide any bounds you can.

#### Part c (3 points)
How would you describe the optimal solution to a non-technical stakeholder?  Explain why your description is sufficient to articulate the optimal solution.

The remaining questions all require you to answer the question and justify your response using the table.  If you cannot provide a precise response without resolving the model, indicate so, and give the best answer you can with the information at hand.

#### Part d) (3 points)
Upon inspection, your floor manager tells you some mice got into the yeast.  You've lost about 5 lbs of yeast that must be discarded.  How do you expect your optimal value to change?

#### Part e) (3 points)
Marketing suggests we can increase the price of ale $3 without affecting demand.  If we did this, what would be the change in objective value?

#### Part f) (3 points)
Your buddy also runs a microbrewery, and they've had a similar problem with mice.  He wants to buy 20 lbs of Malt off you.  You want to quote him a fair price (i.e. you're not trying to make a profit off your friend). What would you sell it for?

### Part g) (3 points)
Assuming demand stays fixed, what is the minimal price increase you'd ask for in Premium Beer to consider producing it?

### Part h) (3 points)
An alternate supplier is willing to sell you 25 lbs of hops at $1 per lb.  Should you take the deal?  Would you expect to make a profit, neither make a profit nor lose money, or lose money if you did? If you can't be sure, explain why.  